In [1]:
import numpy as np
import pickle
import json
import os
import sys
import copy
import sklearn.preprocessing
import models
from sklearn.neighbors import NearestNeighbors
import utils

In [2]:
DATA_DIR = "../../data/fb15k/"
DUMP_FILE = "../dumps/fb15k_distmult_dump_norm.pkl"
MODEL_TYPE = "fb15k"
mining_dir='mined/'

In [3]:
train_data = utils.read_data(os.path.join(DATA_DIR,"train.txt"))
# dev_data = read_data(os.path.join(DATA_DIR,"valid.txt"))
# test_data = read_data(os.path.join(DATA_DIR,"test.txt"))

In [4]:
dump=utils.load_pickle(DUMP_FILE)
dump.keys()

dict_keys(['tail_rel_type', 'entity_type', 'entity_to_id', 'head_rel_type', 'entity_real', 'rel_real', 'relation_to_id'])

In [5]:
model=models.TypedDM(DUMP_FILE)

In [6]:
mapped_train_data = utils.map_data(train_data,dump)
# mapped_dev_data = map_data(dev_data)
# mapped_test_data = map_data(test_data)

In [7]:
entity_to_rel=utils.get_ent_to_rel(mapped_train_data)

In [8]:
rules_1_path=os.path.join(mining_dir,"pruned_rules_1.pkl")
rules_2_path=os.path.join(mining_dir,"pruned_rules_2.pkl")
rules_3_path=os.path.join(mining_dir,"mined_3_ent.pkl")
rules_1=utils.load_pickle(rules_1_path)
rules_2=utils.load_pickle(rules_2_path)
rules_3=utils.load_pickle(rules_3_path)
print(len(rules_1),len(rules_2),len(rules_3))

3869 1836 31394553


In [9]:
def add_relation_body(rules,relation_to_body):
    for rule in rules:
        if rule[1] not in relation_to_body:
            relation_to_body[rule[1]]=[]
        relation_to_body[rule[1]].append(rule[0])
    return relation_to_body

In [10]:
relation_to_body={}
relation_to_body=add_relation_body(rules_1,relation_to_body)
relation_to_body=add_relation_body(rules_2,relation_to_body)
relation_to_body=add_relation_body(rules_3,relation_to_body)
print(len(relation_to_body))

1234


# Length 1 Rules

In [11]:
dict_1=utils.get_r_e1e2_dict(mapped_train_data)

# Length 2 Rules

In [12]:
index_head=utils.get_head_index(mapped_train_data)
dict_2=utils.get_r1r2_e1e2_dict(mapped_train_data,index_head)

## Length 3 Rules

In [13]:
## Gives entity in path for given relation and body

In [14]:
re2_e1=utils.get_re2_e1_dict(mapped_train_data)
e1r_e2=utils.get_e1r_e2_dict(mapped_train_data)


In [15]:
set_mapped_train_data=utils.get_set_mapped_train_data(mapped_train_data)

In [16]:
def solve_3(fact,body,e1r_e2,re2_e1,set_mapped_train_data):
    r1=body[0]
    r2=body[1]
    r3=body[2]
    e1=fact[0]
    e4=fact[2]
    
    key1=(e1,r1)
    key2=(r3,e4)
    
    if key1 not in e1r_e2:
        return ("",-1)
    if key2 not in re2_e1:
        return ("",-1)
    list1=e1r_e2[key1]
    list2=re2_e1[key2]
    
    for e2 in list1:
        for e3 in list2:
            if (e2,r2,e3) in set_mapped_train_data:
                return (body,(e2,e3))
    
    return ("",-1)
    

In [17]:
def get_explanation(fact,relation_to_body,dict_1,dict_2,e1r_e2,re2_e1,set_mapped_train_data):
    pair=(fact[0],fact[2])
    rel=fact[1]
    
    if rel not in relation_to_body:
        return ("",-1)
    bodies=relation_to_body[rel]
    for body in bodies:
        if isinstance(body,int):
            if pair not in dict_1[body]:
                continue
            return (body,-1)
        else:
            if len(body)==2:
                if pair not in dict_2[body]:
                    continue
                return (body,dict_2[body][pair])
            else:
                return solve_3(fact,body,e1r_e2,re2_e1,set_mapped_train_data)                

    return ("",-1)

In [18]:
# get_explanation((453,37,82),relation_to_body,dict_1,dict_2)

In [19]:
triplet_file="../logs/fb15k/turk_test_kl/test_hits_1_id_small.txt"
data=utils.read_data(triplet_file)
np_arr=np.array(utils.map_data(data,dump=dump)).astype(np.int32)
# np_arr=np.loadtxt(triplet_file)

In [20]:
arr=[]
for line in np_arr:
#     print(line)
    fact=(int(line[0]),int(line[1]),int(line[2]))
    arr.append(get_explanation(fact,relation_to_body,dict_1,dict_2,e1r_e2,re2_e1,set_mapped_train_data))
#     print(fact)

In [21]:
# print(arr)
count=0
for x in arr:
    if x[0]!="":
        count+=1
print(count)

167


In [22]:
path=os.path.join(mining_dir,"explanation_123_kl.pkl")
utils.dump_pickle(arr,path)

In [23]:
print(len(np_arr),len(arr))

300 300


In [24]:
lol=utils.load_pickle(path)

In [ ]:
print(len(lol))
print(lol[0:100])